In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
BATCH_SIZE =32
IMAGE_SIZE=256
CHANNEL = 3

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
"../input/plant-village/PlantVillage",
    seed=212,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE)

In [ ]:
class_names=dataset.class_names
n_class=len(dataset.class_names)
print(n_class)

In [ ]:
print(len(dataset))

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

#  **Visualize data**

nan

In [ ]:
plt.figure(figsize=(12,12))
for image_batch , label_batch in dataset.take(1):
    for i in range(15) : 
        ax = plt.subplot(3,5,i+1)
        #convert the array from tensor to numpy array
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")
    

In [ ]:
def get_train_val_test_split(ds,shuffle=True,shuffle_size=100):
    ds_size=len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 12)
    train_size=int(ds_size*0.8)
    val_size=int(ds_size*0.1)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=get_train_val_test_split(dataset)
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

# Data preprocessing 

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])


In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomContrast(0.3)
])

In [ ]:
nan

# BUILD CNN MODEL

In [ ]:
input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNEL)
classifier = Sequential()
#resize and augmentation
classifier.add(resize_and_rescale)
classifier.add(data_augmentation)
# Convolution Step 1
classifier.add(Conv2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=input_shape, activation = 'relu'))
classifier.add(BatchNormalization())
# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Conv2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Conv2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())
# Flattening Step
classifier.add(Flatten())
# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 2048, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1024, activation = 'relu'))
classifier.add(Dropout(0.6))
classifier.add(BatchNormalization())
classifier.add(Dense(units = n_class, activation = 'softmax'))



In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
early_stopping=EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=6, verbose=1, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1,
)
callbacks = [
    early_stopping,
    lr_scheduler    
]
classifier.compile(optimizer=opt,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])


In [ ]:
history = classifier.fit(train_ds ,validation_data = val_ds , batch_size = 32 , epochs = 50,shuffle=True,callbacks = callbacks)


In [ ]:
classifier.summary()

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']



plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
for img, label in test_ds.take(1):
    for i in range(9):
        prediction = classifier.predict(img)
        ax = plt.subplot(3, 3, i+1)
        score = tf.nn.softmax(prediction[i])
        plt.imshow(img[i].numpy().astype('uint8'))
        plt.title('Truth: ' + class_names[label[i]] +
                  "\n" + 'Prediction: ' + class_names[np.argmax(score)])
        plt.axis('off'),
        plt.tight_layout